In [ ]:
import logging

from IPython.core.display import display, HTML

from scripts.main import load_config, get_day_range
from scripts.retrieve_data import retrieve_and_save_data_from_PACS
from scripts.extract_data import load_transform_and_save_data_from_files
from scripts.create_report import create_report, get_report_type

import pandas as pd
import os
import matplotlib.pyplot as plt
from matplotlib.cbook import get_sample_data
from matplotlib.patches import FancyBboxPatch
from random import random
from datetime import date, timedelta
from datetime import datetime as dt


import pandas as pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

%load_ext autoreload
%reload_ext autoreload
%autoreload 2

# set the width of the notebook
display(HTML("<style>.container { width:95% !important; }</style>"))

### Create the report for ~4 years

In [ ]:
config = load_config()
config['main']['start_date'] = '2016-01-01'
config['main']['end_date'] = '2019-12-06'
create_report(config)

### Create the report for a year

In [ ]:
config = load_config()
config['main']['start_date'] = '2019-01-01'
config['main']['end_date'] = '2019-12-06'
create_report(config)

### Create the report for 6 months

In [ ]:
config = load_config()
config['main']['start_date'] = '2019-01-01'
config['main']['end_date'] = '2019-06-30'
create_report(config)

### Create the report for 4 months

In [ ]:
config = load_config()
config['main']['start_date'] = '2019-03-01'
config['main']['end_date'] = '2019-06-30'
create_report(config)

### Create the report for a month

In [ ]:
config = load_config()
config['main']['start_date'] = '2019-03-04'
config['main']['end_date'] = '2019-03-29'
create_report(config)

### Create the report for three weeks

In [ ]:
config = load_config()
config['main']['start_date'] = '2019-03-04'
config['main']['end_date'] = '2019-03-22'
create_report(config)

### Create the report for two weeks

In [ ]:
config = load_config()
config['main']['start_date'] = '2019-03-04'
config['main']['end_date'] = '2019-03-15'
create_report(config)

### Create the report for a week

In [ ]:
config = load_config()
config['main']['start_date'] = '2019-03-04'
config['main']['end_date'] = '2019-03-08'
create_report(config)